In [445]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn  
import seaborn

In [446]:
%matplotlib inline

In [447]:
data = pd.read_csv("cereals.csv")
data.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups
0,100%_Bran,N,C,70,4,1,130,10,5,6,280,25,3,1,0.33
1,100%_Natural_Bran,Q,C,120,3,5,15,2,8,8,135,0,3,1,1.00
2,All-Bran,K,C,70,4,1,260,9,7,5,320,25,3,1,0.33
3,All-Bran_with_Extra_Fiber,K,C,50,4,0,140,14,8,0,330,25,3,1,0.50
4,Almond_Delight,R,C,110,2,2,200,1,14,8,-1,25,3,1,0.75


Sugar, potass, carbo have negative values, meaning missing data

In [448]:
for x in data:
    if x not in ["name", "mfr", "type", "shelf", "vitamins"]:
        data[x] = (data[x] / data.cups).round()

In [449]:
data = data.replace(to_replace=-1, value=np.nan)

In [450]:
data = data.interpolate()
data.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups
0,100%_Bran,N,C,212,12,3,394,30,15,18,848.0,25,3,3,1
1,100%_Natural_Bran,Q,C,120,3,5,15,2,8,8,135.0,0,3,1,1
2,All-Bran,K,C,212,12,3,788,27,21,15,970.0,25,3,3,1
3,All-Bran_with_Extra_Fiber,K,C,100,8,0,280,28,16,0,660.0,25,3,2,1
4,Almond_Delight,R,C,147,3,3,267,1,19,11,376.5,25,3,1,1


In [451]:
# now we need to normalize the data

In [452]:
data.potass = data.potass / 1000
data.sodium = data.sodium / 1000


In [453]:
data[data.vitamins == 0]

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups
1,100%_Natural_Bran,Q,C,120,3,5,0.015,2,8.0,8,0.135,0,3,1,1
20,Cream_of_Wheat_(Quick),N,H,100,3,0,0.080,1,21.0,0,0.175,0,2,1,1
54,Puffed_Rice,Q,C,50,1,0,0.000,0,13.0,0,0.015,0,3,0,1
55,Puffed_Wheat,Q,C,50,2,0,0.000,1,10.0,0,0.050,0,3,0,1
57,Quaker_Oatmeal,Q,H,149,7,3,0.000,4,23.5,14,0.164,0,1,1,1
63,Shredded_Wheat,N,C,80,2,0,0.000,3,16.0,0,0.095,0,1,1,1
64,Shredded_Wheat_'n'Bran,N,C,134,4,0,0.000,6,28.0,0,0.209,0,1,1,1
65,Shredded_Wheat_spoon_size,N,C,134,4,0,0.000,4,30.0,0,0.179,0,1,1,1


I don't think these cereals actually have no vitamins in them, especially something like shredded wheat. Also, 100 vitamins seems unlikely. I'm going to delete the vitamins column as I believe most cereals have similar added vitamins. I also don't believe we ever categorize cereals by their vitamin content.

In [454]:
data = data.drop('vitamins', axis=1)

In [455]:
data = data.drop('weight', axis=1)

I also don't want the particular manufacturer influencing the clustering of the cereals. Afterwards, it will be interesting to analyze whether manufacturers tend to produce similar types of cereals without this actually affecting the clustering.

In [456]:
data = data.drop('mfr', axis=1)

In [457]:
data = data.replace(to_replace="H", value=1)
data = data.replace(to_replace="C", value=0)

Gave hot and cold cereals an integer value

In [458]:
data.index = data.pop("name")

In [460]:
data.head()

,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,shelf,cups
name,,,,,,,,,,,
100%_Bran,0,212,12,3,0.394,30,15,18,0.8480,3,1
100%_Natural_Bran,0,120,3,5,0.015,2,8,8,0.1350,3,1
All-Bran,0,212,12,3,0.788,27,21,15,0.9700,3,1
All-Bran_with_Extra_Fiber,0,100,8,0,0.280,28,16,0,0.6600,3,1
Almond_Delight,0,147,3,3,0.267,1,19,11,0.3765,3,1


In [461]:
from sklearn.decomposition import PCA


/Users/MattKasle/Matt/.direnv/python-3.5.0/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:
